# Parte 2 (Detector Forense)

**Objetivo:** Analizar todas las imágenes del dataset y detectar cuál contiene un mensaje oculto en el LSB.
Buscaremos el patrón `"FLAG{"` y, cuando aparezca, extraeremos el mensaje completo.

# Importaciones y configuración

In [7]:
from PIL import Image
import numpy as np
import os
import time

FOLDER = "dataset_images"
HEADER = "FLAG{"
END_DELIMITER = "1111111111111110"  # Debe coincidir con el generador (16 bits)


## 1) Extracción de LSB

La idea es recorrer los píxeles de la imagen (RGB) y extraer el bit menos significativo (LSB)
de cada canal: R, G y B.

- Si `num_bits=None`, extraemos *todos* los bits disponibles.
- Si `num_bits` es un número, extraemos solo los primeros `num_bits` (esto nos permite optimizar).


In [8]:
def extract_lsb(image_path: str, num_bits: int | None = None) -> str:
    """
    Extrae bits LSB de una imagen.
    
    Args:
        image_path: ruta del PNG
        num_bits: número de bits a extraer (None = todos)

    Returns:
        bitstring, por ejemplo "010101..."
    """
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)

    # Aplanamos canales: [R,G,B,R,G,B,...]
    flat = data.reshape(-1)

    if num_bits is None:
        relevant = flat
    else:
        relevant = flat[:num_bits]

    # LSB de cada valor (0 o 1) -> lo convertimos a string
    bits = (relevant & 1).astype(np.uint8)
    return "".join(bits.astype(str))


## 2) Convertir bits a texto (ASCII) y cortar por delimitador

Agrupamos en bytes (8 bits) -> convertimos a caracteres.
Paramos cuando encontramos el delimitador final (16 bits).

**Nota:** Esta función es la que usarás para reconstruir el mensaje completo en la imagen correcta.


In [9]:
def binary_to_text(bits: str, end_delimiter: str = END_DELIMITER) -> str:
    """
    Convierte bitstring a texto ASCII y corta al encontrar delimitador.
    """
    # Cortamos en el primer delimitador si aparece
    end_idx = bits.find(end_delimiter)
    if end_idx != -1:
        bits = bits[:end_idx]

    chars = []
    # Convertimos byte a byte
    for i in range(0, len(bits) - 7, 8):
        byte = bits[i:i+8]
        chars.append(chr(int(byte, 2)))
    return "".join(chars)


## 3) Búsqueda Naive (Brute Force)

Para cada imagen:
1. Extraemos TODOS los bits LSB.
2. Reconstruimos el texto completo.
3. Buscamos el patrón `"FLAG{"`.

Esto es costoso porque lee y procesa toda la imagen siempre.


In [10]:
def search_brute_force(folder: str, header: str = HEADER):
    """
    Busca el mensaje con enfoque brute force.
    Devuelve (filename, message) si lo encuentra, o (None, None).
    """
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])

    for fname in files:
        path = os.path.join(folder, fname)

        bits = extract_lsb(path, num_bits=None)      # extrae TODO
        text = binary_to_text(bits)                  # reconstruye

        if header in text:
            return fname, text

    return None, None


## 4) Búsqueda Optimizada (Early Termination)

Idea:
- No hace falta reconstruir toda la imagen para descartar la mayoría.
- Solo extraemos los bits necesarios para verificar el header `"FLAG{"`.

Cálculo:
- header `"FLAG{"` tiene 5 caracteres -> 5 bytes -> 40 bits.
- Para reconstruir 40 bits necesitamos extraer exactamente esos bits del LSB.

Proceso:
1. Extraer 40 bits -> convertir a texto -> comparar con `"FLAG{"`.
2. Si coincide, entonces sí extraemos todos los bits y reconstruimos el mensaje completo.
3. Si no coincide, pasamos a la siguiente imagen.


In [11]:
def search_optimized(folder: str, header: str = HEADER):
    """
    Búsqueda optimizada: primero verifica header, luego extrae completo solo si coincide.
    Devuelve (filename, message) o (None, None).
    """
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])

    header_bits_len = len(header) * 8  # 5 chars * 8 = 40 bits

    for fname in files:
        path = os.path.join(folder, fname)

        # 1) Extraer solo lo mínimo para comprobar header
        bits_prefix = extract_lsb(path, num_bits=header_bits_len + len(END_DELIMITER))
        # Convertimos solo lo necesario a texto
        # (en realidad con 40 bits basta, pero añadir el delimitador aquí no estorba)
        text_prefix = binary_to_text(bits_prefix, end_delimiter=END_DELIMITER)

        if text_prefix.startswith(header):
            # 2) Ahora sí: extraemos todo para reconstruir mensaje completo
            bits_all = extract_lsb(path, num_bits=None)
            text_full = binary_to_text(bits_all, end_delimiter=END_DELIMITER)
            return fname, text_full

    return None, None


## 5) Ejecutar detector y medir tiempos

Vamos a ejecutar:
- Brute force
- Optimizado

y comparar tiempo y resultado encontrado.


In [12]:
print("="*60)
print("LSB FORENSIC ANALYSIS — PARTE 2")
print("="*60)

# Brute Force
t0 = time.time()
bf_file, bf_msg = search_brute_force(FOLDER)
t1 = time.time()

print("\n[BRUTE FORCE]")
print(f"Tiempo: {t1 - t0:.4f} s")
print(f"Archivo encontrado: {bf_file}")
if bf_msg:
    print(f"Mensaje: {bf_msg}")

# Optimizado
t2 = time.time()
op_file, op_msg = search_optimized(FOLDER)
t3 = time.time()

print("\n[OPTIMIZADO]")
print(f"Tiempo: {t3 - t2:.4f} s")
print(f"Archivo encontrado: {op_file}")
if op_msg:
    print(f"Mensaje: {op_msg}")



LSB FORENSIC ANALYSIS — PARTE 2

[BRUTE FORCE]
Tiempo: 1.3941 s
Archivo encontrado: img_064.png
Mensaje: FLAG{LUCIACANTOSBURGOS}

[OPTIMIZADO]
Tiempo: 0.0465 s
Archivo encontrado: img_064.png
Mensaje: FLAG{LUCIACANTOSBURGOS}


## Si NO aparece el mensaje...

Las causas más comunes son:
- El `END_DELIMITER` no coincide exactamente con el del generador.
- El mensaje se escribió con un formato distinto (por ejemplo espacios o minúsculas).
- En el generador, el FLAG no empezaba exactamente por `"FLAG{"`.

Si pasa, lo depuramos revisando:
1) el header extraído del prefix  
2) que el delimitador se detecta bien  
3) que el orden de recorrido (R,G,B) coincide en ocultación y extracción